** download data in kaggle**

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sepide242477","key":"d1ec2f06a5ab676086672c24f346ca43"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets list

ref                                                                title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                           Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23          14213        409  1.0              
zusmani/petrolgas-prices-worldwide                                 Petrol/Gas Prices Worldwide                          10KB  2022-06-24 01:25:33           1585         83  1.0              
ruchi798/data-science-job-salaries                                 Data Science Job Salaries                             7KB  2022-06-15 08:59:12           2768         90  1.0              
devansodariya/student-performance-data       

In [5]:
!kaggle datasets download -d sakshaymahna/kittiroadsegmentation

 94% 286M/305M [00:01<00:00, 224MB/s]
100% 305M/305M [00:01<00:00, 174MB/s]


In [ ]:
! unzip '/content/kittiroadsegmentation.zip'

#  Creat Fully Convolution Network

In [ ]:
!pip install segmentation-models

In [8]:
import tensorflow as tf 

In [9]:
from tensorflow.keras.layers import Conv2D,UpSampling2D,Input,Concatenate
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import segmentation_models as sm
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU

Segmentation Models: using `keras` framework.


In [10]:
#FCN model
inputs=Input(shape=(None,None,3))
vgg=VGG16(weights='imagenet',include_top=False,input_tensor=inputs)

c1=vgg.get_layer('block3_pool').output
c2=vgg.get_layer('block4_pool').output
c3=vgg.get_layer('block5_pool').output

u1=UpSampling2D((2,2),interpolation='bilinear')(c3)
d1=Concatenate()([u1,c2])
d1=Conv2D(256,1,activation='sigmoid')(d1)

u2=UpSampling2D((2,2),interpolation='bilinear')(d1)
d2=Concatenate()([u2,c1])
d2=Conv2D(256,1,activation='sigmoid')(d2)

u3=UpSampling2D((8,8),interpolation='bilinear')(d2)
outputs=Conv2D(1,1,activation='sigmoid')(u3)

model=Model(inputs,outputs)

58900480/58889256 [==============================] - 0s 0us/step


In [44]:
meam_iou=MeanIoU(2)
optimizer = tf.keras.optimizers.Adam()
Loss=sm.losses.DiceLoss()
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=[meam_iou,'acc'])

## load data

In [52]:
import os 
import cv2 
import numpy as np
import matplotlib.pyplot as plt 
from keras.preprocessing.image import img_to_array,load_img
import seaborn as sns 

In [25]:
maskes=[]
images=[]

In [26]:
image_path='/content/data/training/image_2'
mask_path='/content/data/training/gt_image_2'
image_file=os.listdir(image_path)
mask_file=os.listdir(mask_path) 
for name in os.listdir(image_path):
  path=os.path.join(image_path,name)
  image=load_img(path,target_size=(224,224,3))
  image=img_to_array(image)
  
  images.append(image)

for name in os.listdir(mask_path):
  path=os.path.join(mask_path,name)
  mask=load_img(path,target_size=(224,224,3))
  mask=img_to_array(mask)
  mask=H.ben(mask)
  label=H.getSegmentationArr(mask)
  maskes.append(label)

In [27]:
images=np.array(images)
maskes=np.array(maskes)

In [28]:
maskes=maskes[:289,:,:,:]

# fit model

In [45]:
he=model.fit(images,maskes,batch_size=32,epochs=10,validation_split=0.25)

Epoch 1/10
7/7 [==============================] - 6s 698ms/step - loss: 0.3752 - mean_io_u_5: 0.4935 - acc: 0.9871 - val_loss: 0.4980 - val_mean_io_u_5: 0.4912 - val_acc: 0.9825
Epoch 2/10
7/7 [==============================] - 5s 659ms/step - loss: 0.3613 - mean_io_u_5: 0.4935 - acc: 0.9871 - val_loss: 0.4809 - val_mean_io_u_5: 0.4912 - val_acc: 0.9825
Epoch 3/10
7/7 [==============================] - 4s 637ms/step - loss: 0.3490 - mean_io_u_5: 0.4935 - acc: 0.9871 - val_loss: 0.4633 - val_mean_io_u_5: 0.4912 - val_acc: 0.9825
Epoch 4/10
7/7 [==============================] - 4s 644ms/step - loss: 0.3361 - mean_io_u_5: 0.4935 - acc: 0.9871 - val_loss: 0.4457 - val_mean_io_u_5: 0.4912 - val_acc: 0.9825
Epoch 5/10
7/7 [==============================] - 4s 650ms/step - loss: 0.3228 - mean_io_u_5: 0.4935 - acc: 0.9871 - val_loss: 0.4281 - val_mean_io_u_5: 0.4912 - val_acc: 0.9825
Epoch 6/10
7/7 [==============================] - 5s 655ms/step - loss: 0.3103 - mean_io_u_5: 0.4935 - acc: 0.

In [46]:
model.save('segmentation_model.h5')